In [7]:
# !pip install lightning

In [8]:
import pytorch_lightning as pl
from torch.utils.data import DataLoader
import albumentations as A
import torchvision.transforms as T
import albumentations.pytorch as pytorch
import albumentations as albu

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchmetrics import Accuracy, JaccardIndex, FBetaScore
from typing import Any, Union

In [10]:
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger

import segmentation_models_pytorch as smp
import warnings

In [11]:

from torch.utils.data import Dataset

import pandas as pd
import numpy as np

import os

from PIL import Image

from segmentation_models_pytorch.utils import metrics

from segmentation_models_pytorch.losses import FocalLoss, DiceLoss, JaccardLoss

import re

import onnx
import onnxruntime

import pandas as pd
%matplotlib inline
# %matplotlib widget

import matplotlib as mpl
mpl.rc('font',family='Charter')

import matplotlib.pyplot as plt

import shutil

torch.manual_seed(42)

In [12]:
class ThermalDataset(Dataset):
    def __init__(self,
                 stage: str,
                 images_path: str,
                 augmentation: Any,
                 preprocessing: Any,
                 shuffle: bool = True,
                 random_state: int = 42):

        self.__attribute_checking(images_path,
                                  stage, shuffle, random_state)

        self.images_path = images_path

        self.augmentation = augmentation
        self.preprocessing = preprocessing

        self.stage = stage
        self.shuffle = shuffle
        self.random_state = random_state
        self.total_len = None
        self._images, self._masks = self.__create_dataset()

    @staticmethod
    def __type_checking(images_path: str,
                        stage: str, shuffle: bool,
                        random_state: int) -> None:
        
        assert isinstance(images_path, str)
        assert isinstance(stage, str)
        assert isinstance(shuffle, bool)
        assert isinstance(random_state, int)


    @staticmethod
    def __path_checking(images_path: str) -> None:
        assert os.path.isdir(images_path)

    @staticmethod
    def __stage_checking(stage: str) -> None:
        assert stage in ["train", "test", "val"]

    @classmethod
    def __attribute_checking(cls, images_path: str,
                             stage: str,
                             shuffle: bool,
                             random_state: int) -> None:

        cls.__type_checking(images_path=images_path,
                            stage=stage,
                            shuffle=shuffle,
                            random_state=random_state)

        cls.__path_checking(images_path=images_path)

        cls.__stage_checking(stage=stage)

    def __create_dataset(self) -> dict:
        dict_paths = {
            "image": [],
            "mask": []
        }

        images_path = self.__split_data(self.stage)

        for image_name in os.listdir(images_path):
            dict_paths["image"].append(os.path.join(images_path,image_name))
            dict_paths["mask"].append(os.path.join(os.path.dirname(images_path),'masks',image_name.replace('_NIR_SWIR','_mask')))

        dataframe = pd.DataFrame(
            data=dict_paths,
            index=np.arange(0, len(dict_paths["image"]))
        )
        self.total_len = len(dataframe)
        data_dict = {self.stage: (dataframe["image"].values,dataframe["mask"].values)}

        return data_dict[self.stage]

    def __split_data(self, stage: str) -> str:
        return os.path.join(self.images_path,stage,'images')

    def __len__(self) -> int:
        return self.total_len

    def __getitem__(self, idx) -> tuple:

        image = Image.open(self._images[idx])
        mask = Image.open(self._masks[idx])
        
        image = np.array(image)

        ### FOR FOCAL LOSS
        mask = mask.convert('L') # This ensures that the label only have 1 band, which is necessary for binary classification
        mask = np.array(mask)[:,:,np.newaxis]
        
        mask = np.divide(mask,255).astype('float32') #Masks need to be 0-1 values
        
        # # apply augmentation
        if self.augmentation:
            sample = self.augmentation(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
        
        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']

        return image, mask


In [13]:
def get_training_augmentation():
    train_transform = [

        albu.HorizontalFlip(p=0.5),
        albu.VerticalFlip(p=0.5),

    ]
    return albu.Compose(train_transform)

def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')

def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform
    Args:
        preprocessing_fn (callbale): data normalization function 
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose
    
    """
    
    _transform = [
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=to_tensor, mask=to_tensor),
    ]
    return albu.Compose(_transform)

In [14]:
class ThermalDataModule(pl.LightningDataModule):
    def __init__(self,images_path: str,
                 augmentation: Union[T.Compose, A.Compose],
                 preprocessing: Any,
                 batch_size: int = 5,
                 num_workers: int = os.cpu_count(),
                 seed: int = 42):
        super().__init__()
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.images_path = images_path
        self.data_train = None
        self.data_val = None
        self.data_test = None
        self.data_predict = None
        self.seed = seed

        self.train_augmentation = augmentation
        self.eval_augmentation = augmentation
        self.preprocessing = preprocessing


    def setup(self, stage: str = None) -> None:
        self.data_train = ThermalDataset(
            images_path=self.images_path,
            augmentation=self.train_augmentation,
            preprocessing=self.preprocessing,
            stage="train",
            shuffle=True,
            random_state=self.seed
            )

        self.data_val = ThermalDataset(
            images_path=self.images_path,
            augmentation=self.eval_augmentation,
            preprocessing=self.preprocessing,
            stage="val",
            shuffle=True,
            random_state=self.seed
            )

        self.data_test = ThermalDataset(
            images_path=self.images_path,
            augmentation=self.eval_augmentation,
            preprocessing=self.preprocessing,
            stage="test",
            shuffle=True,
            random_state=self.seed
            )

        self.data_predict = ThermalDataset(
            images_path=self.images_path,
            augmentation=self.eval_augmentation,
            preprocessing=self.preprocessing,
            stage="test",
            shuffle=True,
            random_state=self.seed
            )

    def train_dataloader(self) -> DataLoader:
        return DataLoader(
            dataset=self.data_train,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=True
        )

    def val_dataloader(self) -> DataLoader:
        return DataLoader(
            dataset=self.data_val,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=False
        )

    def test_dataloader(self) -> DataLoader:
        return DataLoader(
            dataset=self.data_test,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=False
        )

    def predict_dataloader(self) -> DataLoader:
        return DataLoader(
            dataset=self.data_predict,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=False
        )

In [15]:
# class ThermalModel(pl.LightningModule):
#     def __init__(self,
#                  model: nn.Module,
#                  loss_fn: Any,
#                  optim_dict: dict = None,
#                  lr: float = None,
#                  num_classes: int = 1):
#         super().__init__()
#         self.save_hyperparameters(ignore=['model','loss_fn'])

#         self.num_classes = num_classes
#         self.model = model
#         # self.criterion = nn.CrossEntropyLoss()
#         self.criterion = loss_fn
#         self.optim_dict = optim_dict
#         self._device = "cuda" if torch.cuda.is_available else "cpu"

#         self.step_outputs = {
#             "loss": [],
#             "accuracy": [],
#             "jaccard_index": [],
#             "fbeta_score": [],
#             "IoU": []
#         }

#         self.metrics = {
#             "accuracy": Accuracy(task="binary",
#                                  threshold=0.5,
#                                  num_classes=num_classes,
#                                  validate_args=True,
#                                  ignore_index=None,
#                                  average="micro").to(self._device),

#             "jaccard_index": JaccardIndex(task="binary",
#                                           threshold=0.5,
#                                           num_classes=num_classes,
#                                           validate_args=True,
#                                           ignore_index=None,
#                                           average="macro").to(self._device),

#             "fbeta_score": FBetaScore(task="binary",
#                                       beta=1.0,
#                                       threshold=0.5,
#                                       num_classes=num_classes,
#                                       average="micro",
#                                       ignore_index=None,
#                                       validate_args=True).to(self._device),

#             "IoU": metrics.IoU()
#         }

#     def forward(self, x):
#         return self.model(x)

#     def shared_step(self, batch, stage: str) -> torch.Tensor:
#         x, y = batch
#         x, y = x.to(self._device),y.to(self._device)

#         assert x.ndim == 4
#         assert x.max() <= 3 and x.min() >= -3 
#         assert y.ndim == 4
#         assert y.max() <= 1 and y.min() >= 0

#         logits = self.forward(x.to(torch.float32))
        

#         # activated = F.softmax(input=logits, dim=1)
#         # predictions = torch.argmax(activated, dim=1)

#         predictions = torch.round(torch.sigmoid(logits))
#         # predictions = logits
        
#         loss = self.criterion(logits, y)
        
#         accuracy = self.metrics["accuracy"](predictions, y)
#         jaccard_index = self.metrics["jaccard_index"](predictions, y)
#         fbeta_score = self.metrics["fbeta_score"](predictions, y)
#         IoU_score = self.metrics["IoU"](predictions, y)

#         # print(f'stage: {stage}')
#         # print(f'Jaccard: {jaccard_index.dtype}')
#         # print(f'loss: {loss.dtype}')

#         self.step_outputs["loss"].append(loss)
#         self.step_outputs["accuracy"].append(accuracy)
#         self.step_outputs["jaccard_index"].append(jaccard_index)
#         self.step_outputs["fbeta_score"].append(fbeta_score)
#         self.step_outputs["IoU"].append(IoU_score)


#         self.log(f'{stage}_loss'   , loss          , prog_bar=True , on_step=False , on_epoch=True)
#         # self.log(f'{stage}_acc'    , accuracy      , prog_bar=True , on_step=False , on_epoch=True)
#         # self.log(f'{stage}_jaccard', jaccard_index , prog_bar=True , on_step=False , on_epoch=True)
#         self.log(f'{stage}_fbeta'  , fbeta_score   , prog_bar=True , on_step=False , on_epoch=True)
#         self.log(f'{stage}_IoU'    , IoU_score     , prog_bar=True , on_step=False , on_epoch=True)
        
#         return loss

#     def shared_epoch_end(self, stage: Any):
#         loss = torch.mean(torch.tensor([
#             loss for loss in self.step_outputs["loss"]
#         ]))

#         accuracy = torch.mean(torch.tensor([
#             accuracy for accuracy in self.step_outputs["accuracy"]
#         ]))

#         jaccard_index = torch.mean(torch.tensor([
#             jaccard_index for jaccard_index in self.step_outputs["jaccard_index"]
#         ]))

#         print(f'stage: {stage}')
#         print(f'jaccard: {self.step_outputs["jaccard_index"]}')
#         print(f'Result: {jaccard_index}')


#         fbeta_score = torch.mean(torch.tensor(
#             [fbeta_score for fbeta_score in self.step_outputs["fbeta_score"]
#              ]))

#         IoU_score = torch.mean(torch.tensor(
#                 [IoU_score for IoU_score in self.step_outputs["IoU"]
#                  ]))

#         for key in self.step_outputs.keys():
#             self.step_outputs[key].clear()

#         metrics = {
#             f"{stage}_loss": loss,
#             f"{stage}_accuracy": accuracy,
#             f"{stage}_jaccard_index": jaccard_index,
#             f"{stage}_fbeta_score": fbeta_score,
#             f"{stage}_IoU": IoU_score
#         }
#         self.log_dict(metrics, prog_bar=True)

#     def training_step(self, batch: Any, batch_idx: Any):
#         return self.shared_step(batch=batch, stage="train")

#     def on_train_epoch_end(self) -> None:
#         return self.shared_epoch_end(stage="train")

#     def validation_step(self, batch: Any, batch_idx: Any):
#         return self.shared_step(batch=batch, stage="val")

#     def on_validation_epoch_end(self) -> None:
#         return self.shared_epoch_end(stage="val")

#     def test_step(self, batch: Any, batch_idx: Any):
#         return self.shared_step(batch=batch, stage="test")

#     def on_test_epoch_end(self) -> None:
#         return self.shared_epoch_end(stage="test")

#     def predict_step(self, batch: Any, batch_idx: int, dataloader_idx: int = 0):
#         x, y = batch

#         assert x.ndim == 4
#         assert x.max() <= 3 and x.min() >= -3
#         assert y.ndim == 4
#         assert y.max() <= 1 and y.min() >= 0

#         logits = self.forward(x.to(torch.float32))
#         # predictions = logits
#         predictions = torch.round(torch.sigmoid(logits))

#         # activated = F.softmax(input=logits, dim=1)
#         # predictions = torch.argmax(activated, dim=1)

#         return predictions

#     def configure_optimizers(self):
#         optimizer = torch.optim.Adam(
#             params=self.parameters(),
#             lr=self.hparams.lr
#         )

#         scheduler_dict = {
#             "scheduler": torch.optim.lr_scheduler.ReduceLROnPlateau(
#                 optimizer=optimizer,
#                 patience=5
#             ),
#             # "scheduler": pl_bolts.optim.lr_scheduler.LinearWarmupCOsineAnnealingLR(
#             #     optimizer=optimizer,
#             #     warmup_epochs=10,
#             #     max_epochs=30,
#             # ),
#             "interval": "epoch",
#             "monitor": "val_loss"
#         }
        
#         optimization_dictionary = {"optimizer": optimizer, "lr_scheduler": scheduler_dict}
#         return self.optim_dict if self.optim_dict else optimization_dictionary


In [16]:


class Combined_Focal_Dice_Loss(pl.LightningModule):
    '''
    Combined weighted loss between Focal Loss and Dice Loss  
    '''
    def __init__(self,
                 focal_loss_weight: float = 0.5,
                 dice_weight: float = None,
                 log_dice_loss: bool = False):
        
        super(Combined_Focal_Dice_Loss, self).__init__()
        
        self.focal_loss_weight = focal_loss_weight
        self.dice_weight = (1 - focal_loss_weight) if dice_weight is None else dice_weight

        if self.focal_loss_weight + self.dice_weight != 1:
            warnings.warn("Sum of Focal and Dice loss weights is not 1.0: "
                          f"{self.focal_loss_weight:.2f} + {self.dice_weight:.2f} = "
                          f"{self.focal_loss_weight + self.dice_weight:.2f}")

        self.log_dice_loss = log_dice_loss


    # def dice_score(y_pred, y_true, eps=1e-15, smooth=1.):
    #     intersection = (y_pred * y_true).sum()
    #     union = y_pred.sum() + y_true.sum()
    #     return (2. * intersection + smooth) / (union + smooth + eps)


    def forward(self, y_pred, y_true):

        focal_loss_fn = FocalLoss(mode= 'binary')
        focal_loss_fn.__name__ = 'focal_loss'
        dice_loss_fn = DiceLoss(mode= 'binary',from_logits=True,log_loss=self.log_dice_loss) #Typically Dice use the masks and not logits, that is why from logits is used because y_pred are the logits
        dice_loss_fn.__name__ = 'dice_loss'


        focal_loss = focal_loss_fn(y_pred, y_true) 
        dice_loss = dice_loss_fn(y_pred, y_true) 
        
        # y_pred = torch.sigmoid(y_pred)
        # dice_loss = 1- dice_score(y_pred, y_true)
        # log_dice_loss = -torch.log(dice_score(y_pred, y_true))
        
        loss = self.focal_loss_weight * focal_loss + self.dice_weight * dice_loss

        return loss
    


In [17]:
import cv2


class ThermalModel(pl.LightningModule):
    def __init__(self,
                 model: nn.Module,
                 loss_fn: Any,
                 optim_dict: dict = None,
                 lr: float = None,
                 num_classes: int = 1):
        super().__init__()
        self.save_hyperparameters(ignore=['model','loss_fn'])

        self.num_classes = num_classes
        self.model = model
        # self.criterion = nn.CrossEntropyLoss()
        self.criterion = loss_fn
        self.optim_dict = optim_dict
        self._device = "cuda" if torch.cuda.is_available else "cpu"

        # self.step_outputs = {
        #     "loss": [],
        #     "accuracy": [],
        #     "jaccard_index": [],
        #     "fbeta_score": [],
        #     "IoU": []
        # }

        # self.stage_outputs = {
        #     "train": self.step_outputs,
        #     "val": self.step_outputs,
        #     "test": self.step_outputs
        # }

        self.metrics = {
            "accuracy": Accuracy(task="binary",
                                 threshold=0.5,
                                 num_classes=num_classes,
                                 validate_args=True,
                                 ignore_index=None,
                                 average="micro").to(self._device),

            "jaccard_index": JaccardIndex(task="binary",
                                          threshold=0.5,
                                          num_classes=num_classes,
                                          validate_args=True,
                                          ignore_index=None,
                                          average="macro").to(self._device),

            "fbeta_score": FBetaScore(task="binary",
                                      beta=1.0,
                                      threshold=0.5,
                                      num_classes=num_classes,
                                      average="micro",
                                      ignore_index=None,
                                      validate_args=True).to(self._device),

            "IoU": metrics.IoU()
        }

    def forward(self, x):
        return self.model(x)

    def shared_step(self, batch, stage: str) -> torch.Tensor:
        x, y = batch
        x, y = x.to(self._device),y.to(self._device)

        assert x.ndim == 4
        assert x.max() <= 3 and x.min() >= -3 
        assert y.ndim == 4
        assert y.max() <= 1 and y.min() >= 0

        logits = self.forward(x.to(torch.float32))
        

        # print(x.cpu().detach().numpy().max(),x.cpu().detach().numpy().min())

        # activated = F.softmax(input=logits, dim=1)
        # predictions = torch.argmax(activated, dim=1)
        


        # prob_mask = logits.sigmoid()
        # prob_mask = nn.Sigmoid()(logits)
        # predictions = (prob_mask > 0.5).float()



        # testing  = (torch.softmax(logits, dim=0))
        testing  = (torch.sigmoid(logits))
        predictions = (testing > 0.5).float()



        # print(testing.shape,np.unique(testing))


    
        # print(y.max(),y.min(),np.unique(y.detach().cpu().numpy()))

        
        # predictions = (logits > 0.5).float()
        
        # print(y.dtype,type(y))
        # print(predictions.dtype,type(predictions))

        # print(y.max(),y.min())
        # print(predictions.max(),predictions.min())


        # print('Y:\n',y)
        # print('Predictions:\n',predictions)


        # y_arr = y.cpu().detach().numpy().astype(np.float32)

        # prediction_arr = predictions.cpu().detach().numpy().astype(np.float32)

        # print(type(y_arr),y_arr.dtype)
        # print(type(prediction_arr),prediction_arr.dtype)

        # print(y_arr[0].shape,prediction_arr.shape)

        # cv2.imshow('x',x.cpu().detach().numpy().astype(np.float32)[0][0])
        # # cv2.imshow('y',y.cpu().detach().numpy().astype(np.float32)[0][0])
        # # cv2.imshow('predictions',predictions.detach().cpu().numpy()[0][0])
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()

        

        # predictions = torch.round(torch.sigmoid(logits))
        # predictions = torch.round(logits)

        # tp, fp, fn, tn  = smp.metrics.get_stats(predictions.long(),y.long(),mode = 'binary')

        # print(predictions)
        # print(tp,fp,fn,tn)
        
        loss = self.criterion(logits, y)
        

        # print(F.binary_cross_entropy(predictions,y)) #Works because this doesn't expect logits

        # print(loss)

        accuracy = self.metrics["accuracy"](predictions, y)
        jaccard_index = self.metrics["jaccard_index"](predictions, y)
        fbeta_score = self.metrics["fbeta_score"](predictions, y)
        IoU_score = self.metrics["IoU"](predictions, y)

        # print(f'stage: {stage}')
        # print(f'Jaccard: {jaccard_index.dtype}')
        # print(f'loss: {loss.dtype}')
        
        if torch.any(y!=0): # This avoids problems with the empty masks, that provides or 1 IoU or nan Jaccard Index 

            # self.stage_outputs[stage]["loss"].append(loss)
            # self.stage_outputs[stage]["accuracy"].append(accuracy)
            # self.stage_outputs[stage]["jaccard_index"].append(jaccard_index)
            # self.stage_outputs[stage]["fbeta_score"].append(fbeta_score)
            # self.stage_outputs[stage]["IoU"].append(IoU_score)


            self.log(f'{stage}_loss'   , loss          , prog_bar=True , on_step=False , on_epoch=True)
            self.log(f'{stage}_acc'    , accuracy      , prog_bar=True , on_step=False , on_epoch=True)
            self.log(f'{stage}_jaccard', jaccard_index , prog_bar=True , on_step=False , on_epoch=True)
            self.log(f'{stage}_fbeta'  , fbeta_score   , prog_bar=True , on_step=False , on_epoch=True)
            self.log(f'{stage}_IoU'    , IoU_score     , prog_bar=True , on_step=False , on_epoch=True)

            # self.log(f'{stage}_tp'     , tp            , prog_bar=True , on_step=False , on_epoch=True)
            # self.log(f'{stage}_fp'     , fp            , prog_bar=True , on_step=False , on_epoch=True)
            # self.log(f'{stage}_fn'     , fn            , prog_bar=True , on_step=False , on_epoch=True)
            # self.log(f'{stage}_tn'     , tn            , prog_bar=True , on_step=False , on_epoch=True)

        # else:
            # print(stage,loss,jaccard_index,IoU_score,fbeta_score)
            # for i in range(5):
            #     plt.figure()
            #     plt.subplot(1,2,1)
            #     plt.imshow(predictions.detach().cpu().numpy().squeeze()[i,:,:])
            #     plt.subplot(1,2,2)
            #     plt.imshow(y.detach().cpu().numpy().squeeze()[i,:,:])
            #     plt.show()


        # self.log(f'{stage}_loss'   , loss          , prog_bar=True , on_step=False , on_epoch=True)
        # self.log(f'{stage}_acc'    , accuracy      , prog_bar=True , on_step=False , on_epoch=True)
        # self.log(f'{stage}_jaccard', jaccard_index , prog_bar=True , on_step=False , on_epoch=True)
        # self.log(f'{stage}_fbeta'  , fbeta_score   , prog_bar=True , on_step=False , on_epoch=True)
        # self.log(f'{stage}_IoU'    , IoU_score     , prog_bar=True , on_step=False , on_epoch=True)
        
        # self.optimizers().step()
        # self.lr_schedulers().step()

        
        return loss

        # return {
        #     "loss": loss,
        #     "tp": tp,
        #     "fp": fp,
        #     "fn": fn,
        #     "tn": tn,
        # }

    def shared_epoch_end(self,outputs, stage: Any):

        tp = torch.cat([x["tp"] for x in outputs])
        fp = torch.cat([x["fp"] for x in outputs])
        fn = torch.cat([x["fn"] for x in outputs])
        tn = torch.cat([x["tn"] for x in outputs])
        loss = torch.cat([x["loss"] for x in outputs])

        loss = torch.mean(torch.tensor([
            loss for loss in self.stage_outputs[stage]["loss"]
        ]))
        
        accuracy = torch.mean(torch.tensor([
            accuracy for accuracy in self.stage_outputs[stage]["accuracy"]
        ]))

        jaccard_index = torch.mean(torch.tensor([
            jaccard_index for jaccard_index in self.stage_outputs[stage]["jaccard_index"]
        ]))

        fbeta_score = torch.mean(torch.tensor(
            [fbeta_score for fbeta_score in self.stage_outputs[stage]["fbeta_score"]
             ]))

        IoU_score = torch.mean(torch.tensor(
                [IoU_score for IoU_score in self.stage_outputs[stage]["IoU"]
                 ]))
        # print(f'stage: {stage}')
        # print(f'Result: {loss,jaccard_index,fbeta_score,IoU_score}')

        metrics = {
            f"{stage}_loss": loss,
            f"{stage}_accuracy": accuracy,
            f"{stage}_jaccard_index": jaccard_index,
            f"{stage}_fbeta_score": fbeta_score,
            f"{stage}_IoU": IoU_score
        }
        # self.log_dict(metrics, prog_bar=True)

        for key in self.stage_outputs[stage].keys():
            
            print(stage,key)
            
            self.stage_outputs[stage][key].clear()


    def training_step(self, batch: Any, batch_idx: Any):
        return self.shared_step(batch=batch, stage="train")

    # def on_train_epoch_end(self) -> None:
    #     return self.shared_epoch_end(stage="train")

    def validation_step(self, batch: Any, batch_idx: Any):
        return self.shared_step(batch=batch, stage="val")

    # def on_validation_epoch_end(self) -> None:
    #     return self.shared_epoch_end(stage="val")

    def test_step(self, batch: Any, batch_idx: Any):
        return self.shared_step(batch=batch, stage="test")

    # def on_test_epoch_end(self) -> None:
    #     return self.shared_epoch_end(stage="test")

    def predict_step(self, batch: Any, batch_idx: int, dataloader_idx: int = 0):
        x, y = batch

        assert x.ndim == 4
        assert x.max() <= 3 and x.min() >= -3
        assert y.ndim == 4
        assert y.max() <= 1 and y.min() >= 0

        logits = self.forward(x.to(torch.float32))
        # predictions = torch.round(logits)
        # predictions = torch.round(torch.sigmoid(logits))
        
        # prob_mask = logits.sigmoid()
        # predictions = (prob_mask > 0.5).float()
        predictions = (logits > 0.5).float()


        # activated = F.softmax(input=logits, dim=1)
        # predictions = torch.argmax(activated, dim=1)

        return predictions

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(
            params=self.parameters(),
            lr=self.hparams.lr
        )

        scheduler_dict = {
            "scheduler": torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer=optimizer,
                patience=5
            ),
            
            # "scheduler": pl_bolts.optimizers.lr_scheduler.LinearWarmupCosineAnnealingLR(
            #     optimizer=optimizer,
            #     warmup_epochs=2,
            #     max_epochs=3,
            #     eta_min = 0.001
            # ),
            # "interval": "step",
            
            "interval": "epoch",
            "monitor": "val_loss"
        }
        
        optimization_dictionary = {"optimizer": optimizer, "lr_scheduler": scheduler_dict}
        return self.optim_dict if self.optim_dict else optimization_dictionary


In [18]:
def main(callbacks: list,
         model: Union[list, tuple],
         loss_fn: Any,
         augmentation: Any,
         preprocessing: Any,
         logger: Any,
         images_path: str,
         optim_dict: dict,
         min_epochs: int,
         max_epochs: int,
         precision: int
         ) -> None:

    # Trainer
    trainer = pl.Trainer(
        fast_dev_run=False,
        accelerator="auto",
        strategy="auto",
        devices="auto",
        num_nodes=1,
        logger=logger,
        callbacks=callbacks,
        max_epochs=max_epochs,
        min_epochs=min_epochs,
        precision=precision # Mixed precision training
    )

    # Datamodule
    datamodule = ThermalDataModule(
        images_path=images_path,
        augmentation=augmentation,
        preprocessing=preprocessing,
        batch_size=5,
        num_workers=os.cpu_count()
    )

    # LightningModule
    lightning_model = ThermalModel(
        model=model,
        loss_fn=loss_fn,
        optim_dict=optim_dict,
        lr=3e-4
    )

    # Start training
    trainer.fit(model=lightning_model, datamodule=datamodule)

In [19]:
# Run Constants
SEED: int = 42
ACTION: str = "ignore"
DATA_PATH: str = os.path.join(os.getcwd(),'train_dataset')
CHECKPOINT: Any = None
    
# Model Constants
CLASSES = 1
IN_CHANNELS = 3

optim_dict = None

# ENCODER = 'se_resnext50_32x4d'
ENCODER = 'mobilenet_v2'
# ENCODER = 'resnet18'
# ENCODER = 'timm-mobilenetv3_large_100'
ENCODER_WEIGHTS = 'imagenet'
    


ACTIVATION = None
# ACTIVATION = 'sigmoid' # could be None for logits. If used, the sigmoid after the forward function needs to be removed
DEVICE = 'cuda'

min_epochs = 150
max_epochs = 200

n_cpu = os.cpu_count()

model_name = 'Unet'
# model_name = 'DeepLabV3Plus'

model_name_path = os.path.join(os.getcwd(),'models',f'{model_name}_{ENCODER}')
os.makedirs(model_name_path,exist_ok=True)  
run_idx =sum(1 for file in os.listdir(model_name_path) if file.startswith('run'))

model_main_path = os.path.join(model_name_path,f'run_{run_idx}')
os.makedirs(model_main_path,exist_ok=True)

print(model_main_path)

# model_main_path = os.path.join(os.getcwd(),'models',f'{model_name}_{ENCODER}_{run_idx}')
metrics_path = os.path.join(model_main_path,'metrics')
os.makedirs(metrics_path,exist_ok=True)

# model = smp.DeepLabV3Plus(
#     encoder_name=ENCODER, 
#     encoder_weights=ENCODER_WEIGHTS, 
#     classes=1, 
#     activation=ACTIVATION,
# )

model = smp.Unet(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    in_channels = 3,
    classes=CLASSES, 
    activation=ACTIVATION,
)

preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

precision = '16-mixed' # 32 # 32 is the original, and 16 is mixed precission
# precision = 32

loss = FocalLoss(mode= 'binary')
loss.__name__ = 'focal_loss'

# # loss = DiceLoss(mode= 'binary')
# # loss.__name__ = 'dice_loss'

# loss = JaccardLoss(mode= 'binary')
# loss.__name__ = 'jaccard_loss'

# # loss = losses.DiceLoss()
# # loss = losses.JaccardLoss()

# metrics = [
#     metrics.IoU(),
# ]

# optimizer = torch.optim.Adam([ 
#     dict(params=model.parameters(), lr=1e-3),
# ])

augmentation=get_training_augmentation()
preprocessing=get_preprocessing(preprocessing_fn)

/home/cristopher/Documents/SegTHRawS_training/model_training/models/Unet_mobilenet_v2/run_0


In [20]:
# Callbacks
callbacks = [
    ModelCheckpoint(
        dirpath=model_main_path,
        filename=f"{model_name}_{ENCODER}_"+"{epoch}",
        save_top_k=10,
        monitor="val_loss",
        mode="min"
    ),

    EarlyStopping(
        monitor="val_loss",
        min_delta=2e-4,
        patience=8,
        verbose=False,
        mode="min"
    ),

    LearningRateMonitor(
        logging_interval="step"
    )
]

In [21]:
# from lightning.pytorch.loggers import TensorBoardLogger
# logger = TensorBoardLogger(save_dir="./logs", name=model_name)

from lightning.pytorch.loggers import CSVLogger
logger = CSVLogger(f"{model_main_path}/csv_logs", name=f"{model_name}_{ENCODER}")


In [23]:
main(
    callbacks=callbacks,
    model=model,
    loss_fn= loss, #Combined_Focal_Dice_Loss(),
    augmentation=augmentation,
    preprocessing=preprocessing,
    logger=logger,
    images_path=DATA_PATH,
    optim_dict=optim_dict,
    min_epochs=10, #min_epochs,
    max_epochs=10, #max_epochs
    precision=16 
)

Using 16bit None Automatic Mixed Precision (AMP)
Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type      | Params
----------------------------------------
0 | model     | Unet      | 6.6 M 
1 | criterion | FocalLoss | 0     
----------------------------------------
6.6 M     Trainab

Epoch 2:  49%|████▉     | 127/258 [00:04<00:04, 30.77it/s, loss=0.000196, v_num=0, val_loss=0.000114, val_acc=1.000, val_jaccard=0.469, val_fbeta=0.602, val_IoU=0.469, train_loss=0.000238, train_acc=1.000, train_jaccard=0.521, train_fbeta=0.643, train_IoU=0.521]

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir models/current_best_model/version_33

# %tensorboard --logdir logs/DeepLabV3Plus/version_0

In [ ]:

# checkpoints_paths = [os.path.join(model_main_path,checkpoint_path) for checkpoint_path in os.listdir(model_main_path) if checkpoint_path[-5:]=='.ckpt']
# checkpoint_path = max(checkpoints_paths, key=lambda x: int(re.search(r'epoch=(\d+)', x).group(1)))

# for checkpoint in checkpoints_paths:
#     if checkpoint != checkpoint_path:
#         os.remove(checkpoint)


In [ ]:
# ### Perform the testing, c  NEED TO create a function
# trained_model = ThermalModel.load_from_checkpoint(checkpoint_path=checkpoint_path,model=model,loss_fn=loss)
# trained_model.eval();

# trainer = pl.Trainer(
#     fast_dev_run=False,
#     accelerator="auto",
#     strategy="auto",
#     devices="auto",
#     num_nodes=1,
#     logger=logger,
#     callbacks=callbacks,
#     max_epochs=1,
#     min_epochs=1,
#     precision=precision #Mixed precision training
# )

# # Datamodule
# datamodule = ThermalDataModule(
#     images_path=DATA_PATH,
#     augmentation=augmentation,
#     preprocessing=preprocessing,
#     batch_size=5,
#     num_workers=os.cpu_count()
# )

# loss_2 = FocalLoss(mode= 'binary')
# loss_2.__name__ = 'focal_loss'

# # LightningModule
# lightning_model = ThermalModel(
#     model=model,
#     loss_fn=loss_2,
#     optim_dict=optim_dict,
#     lr=3e-4
# )

# test_metrics = trainer.test(model=trained_model,datamodule=datamodule)[0]
# # trainer.predict(model=trained_model,datamodule=datamodule)

In [ ]:
# torch.save(trained_model,os.path.join(model_main_path,'trained_model'))

In [ ]:
# import pandas as pd
# %matplotlib inline
# # %matplotlib widget
# import os

# from matplotlib.ticker import AutoMinorLocator, MultipleLocator

# import matplotlib.pyplot as plt

# model_path = os.path.join(os.getcwd(),'models','current_best_model')

# metric_id_list = []

# for file_name in os.listdir(model_path):
#     if file_name[-4:]=='.csv':
#         fig, ax = plt.subplots(figsize=(7,5))
        
#         csv_file = pd.read_csv(os.path.join(model_path,file_name))
#         metric_name = file_name.replace('_evolution.csv','')
        

#         ax.yaxis.set_minor_locator(AutoMinorLocator(4))

#         if metric_name[:5]=='train':
#             ax.set_xlabel('# epochs')

#             metric_id = metric_name.replace('train_','')    
#             if metric_id not in metric_id_list:
#                 for inner_file_name in os.listdir(model_path):
#                     if inner_file_name[-4:]=='.csv' and inner_file_name.replace('_evolution.csv','')==f'val_{metric_id}':
#                         csv_file_inner = pd.read_csv(os.path.join(model_path,inner_file_name))
                        
#                         csv_values = csv_file['Value']
#                         csv_inner_values = csv_file_inner['Value']
                        
#                         ax.plot(csv_values,color='r',label='Training')
#                         ax.plot(csv_inner_values,color='b',label='Validation')
#                         ax.set_title(f'{metric_id} evolution')
#                         ax.set_ylabel(f'{metric_id}')
#                         plt.annotate('%0.2f' % csv_values[len(csv_values)-1], xy=(len(csv_values)-1, csv_values[len(csv_values)-1]))
#                         plt.annotate('%0.2f' % csv_inner_values[len(csv_inner_values)-1], xy=(len(csv_inner_values)-1, csv_inner_values[len(csv_inner_values)-1]))
#                         metric_id_list.append(metric_id)
#                         break
#             else:
#                 metric_id = None
#                 plt.close(fig)
#         elif metric_name[:3]=='val':
#             ax.set_xlabel('# epochs')
            
#             metric_id = metric_name.replace('val_','')
#             if metric_id not in metric_id_list:
#                 for inner_file_name in os.listdir(model_path):
#                     if inner_file_name[-4:]=='.csv' and inner_file_name.replace('_evolution.csv','')==f'train_{metric_id}':
#                         csv_file_inner = pd.read_csv(os.path.join(model_path,inner_file_name))
                        
#                         csv_values = csv_file['Value']
#                         csv_inner_values = csv_file_inner['Value']

#                         ax.plot(csv_values,color='b',label='Validation')
#                         ax.plot(csv_inner_values,color='r',label='Training')
#                         ax.set_title(f'{metric_id} evolution')
#                         ax.set_ylabel(f'{metric_id}')

#                         plt.annotate('%0.2f' % csv_values[len(csv_values)-1], xy=(len(csv_values)-1, csv_values[len(csv_values)-1]))
#                         plt.annotate('%0.2f' % csv_inner_values[len(csv_inner_values)-1], xy=(len(csv_inner_values)-1, csv_inner_values[len(csv_inner_values)-1]))
                        
#                         metric_id_list.append(metric_id)
#             else:
#                 metric_id = None
#                 plt.close(fig)

#         else:
#             csv_values = csv_file['Value']
#             ax.set_title(f'{metric_name} evolution')
#             ax.set_ylabel(f'{metric_name}')
#             ax.set_xlabel('Time')
#             ax.plot(csv_values,color='g',label='learning rate')
#             plt.annotate('%.0E' % csv_values[0], xy=(0, csv_values[0]))
#             plt.annotate('%.0E' % csv_values[len(csv_values)-1], xy=(len(csv_values)-1, csv_values[len(csv_values)-1]))
#             metric_id = 'lr'
#         # ax.plot(csv_file['Value'])
#         if metric_id:
#             ax.legend()
#             plt.savefig(os.path.join(model_path,metric_id+'_evolution.png'))
#         # break
#         # break
#         # plt.show()
#         # break
#         # plt.savefig(os.path.join(model_path,file_name.replace('.csv','.png')))
        


In [ ]:
# x = torch.randn(1, 3, 256, 256).cpu()
# # model_onnx = trained_model.cpu()
# # model_onnx.eval()

# checkpoint_path = '/home/cristopher/Documents/SegTHRawS_training/model_training/models/Unet_mobilenet_v2_metrics_without_empty_masks/Unet_mobilenet_v2_epoch=149.ckpt'
# trained_model = ThermalModel.load_from_checkpoint(checkpoint_path=checkpoint_path,model=model,loss_fn=loss)

# onnx_model_path = checkpoint_path.replace('.ckpt','.onnx')
# model_onnx = trained_model.cpu()
# model_onnx.eval()

# torch_out = model_onnx(x)
# import warnings
# warnings.filterwarnings(category=FutureWarning,action='ignore')
# warnings.filterwarnings(category=torch.jit.TracerWarning,action='ignore')

# # Export the model
# # torch.onnx.export(model_onnx,                                   # model being run
# #                   x,                                            # model input (or a tuple for multiple inputs)
# #                   onnx_model_path,                              # where to save the model (can be a file or file-like object)
# #                   export_params=True,                           # store the trained parameter weights inside the model file
# #                   opset_version=16,                             # the ONNX version to export the model to
# #                   do_constant_folding=True,                     # whether to execute constant folding for optimization
# #                   input_names = ['input'],                      # the model's input names
# #                   output_names = ['output'],                    # the model's output names
# #                   dynamic_axes={'input' : {0 : 'batch_size'},   # variable length axes
# #                                 'output' : {0 : 'batch_size'}})

# torch.onnx.export(model_onnx,                                   # model being run
#                   x,                                            # model input (or a tuple for multiple inputs)
#                   onnx_model_path,                              # where to save the model (can be a file or file-like object)
#                   export_params=True,                           # store the trained parameter weights inside the model file
#                   opset_version=16,                             # the ONNX version to export the model to
#                   do_constant_folding=True,                     # whether to execute constant folding for optimization
#                   input_names = ['input'],                      # the model's input names
#                   output_names = ['output'])

In [ ]:
# import configparser

# model_info = configparser.ConfigParser()
# model_info.read(os.path.join(onnx_model_path,'info.ini'))

In [ ]:

# onnx_model = onnx.load(onnx_model_path)
# onnx.checker.check_model(onnx_model)


# ort_session = onnxruntime.InferenceSession(onnx_model_path, providers=["CPUExecutionProvider"])

# def to_numpy(tensor):
#     return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# # compute ONNX Runtime output prediction
# ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
# ort_outs = ort_session.run(None, ort_inputs)

# # compare ONNX Runtime and PyTorch results
# np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

# print("Exported model has been tested with ONNXRuntime, and the result looks good!")

In [ ]:
# metrics_version_folder_path = os.path.join(model_main_path,'csv_logs',f'{model_name}_{ENCODER}')


# version_files = os.listdir(metrics_version_folder_path)

# version_files.sort(key=lambda x: os.path.getmtime(os.path.join(metrics_version_folder_path,x)),reverse=True)

# metrics_df = pd.read_csv(os.path.join(metrics_version_folder_path,version_files[0],'metrics.csv'))
# metrics_df

In [ ]:
# df = metrics_df.copy()

# selected_rows = df.dropna(subset=['epoch'],axis=0)
# input_epoch = df['epoch'].max()


# #Move the test values from epoch 0 to the epoch of the model 
# selected_rows.loc[df['test_IoU'].notna(),'epoch'] = input_epoch

# combined_df = pd.DataFrame(columns=selected_rows.columns)

# for _, group in selected_rows.groupby('epoch'):
#     if len(group)==2:

#         row1 = group.iloc[0]
#         row2 = group.iloc[1]

#         # print('Row 1: ',row1)
#         # print('Row 2: ',row2)
        
#         combined_row = pd.DataFrame({
#             col: [row1[col] if pd.notna(row1[col]) else row2[col]] for col in df.columns
#         })

#         combined_df = pd.concat([combined_df,combined_row],ignore_index=True)
#     elif len(group)==3:
#         row1 = group.iloc[0]
#         row2 = group.iloc[1]
#         row3 = group.iloc[2]

#         # print('Row 1: ',row1)
#         # print('Row 2: ',row2)
        
#         combined_row = pd.DataFrame({
#             col: [row1[col] if pd.notna(row1[col]) else row2[col] if pd.notna(row2[col]) else row3[col]] for col in df.columns
#         })

#         combined_df = pd.concat([combined_df,combined_row],ignore_index=True)

# csv_name = os.path.basename(checkpoint_path).replace('ckpt','csv')

# metrics_csv_path = os.path.join(metrics_path,f'metrics_{csv_name}   ')

# plots_path = os.path.join(os.path.dirname(metrics_csv_path),'plots')
# os.makedirs(plots_path,exist_ok=True)

# combined_df.to_csv(metrics_csv_path,index=False)

# fig, ax = plt.subplots()

# ax.plot(metrics_df['lr-Adam'].dropna(),color='g')
# ax.set_xlabel('# epochs')
# ax.set_ylabel(f'Lr evolution')
# ax.set_title(f'Lr evolution',fontname="Charter",weight='bold')

# plt.yscale('log')
# plt.savefig(os.path.join(plots_path,'Lr_evolution.png'))

# # ax.plot(train_values,color='b',label=f'Validation {train_values[len(train_values)-1]:.3E}')
# plt.gca().yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter("{x:.3e}"))



# shutil.rmtree(os.path.join(metrics_version_folder_path,version_files[0])) #Delete the old metrics file


In [ ]:
# metrics_df_test = pd.read_csv('/home/cristopher/Documents/SegTHRawS training/model_training/models/Unet_mobilenet_v2_metrics_without_empty_masks/metrics.csv')
# # plt.plot(metrics_df_test['lr-Adam'].dropna())
# # plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))

# # # ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
# # # plt.yscale('log')

# # plt.gca().yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter("{x:.3e}"))
# # plt.set_xlim(0,100)

# fig, ax = plt.subplots()

# ax.plot(metrics_df_test['lr-Adam'].dropna())
# ax.set_xlabel('# epochs')
# ax.set_ylabel(f'Lr')

# plt.yscale('log')

# # ax.plot(train_values,color='b',label=f'Validation {train_values[len(train_values)-1]:.3E}')
# # ax.set_xlabel('# epochs')
# # ax.set_ylabel(f'{metric_id[0].upper()+metric_id[1:]}')

# # ax.set_title(f'{metric_id[0].upper()+metric_id[1:]} evolution',fontname="Charter")
            

In [ ]:


# metric_id_list = []



# csv_file = pd.read_csv(os.path.join(metrics_csv_path))

# columns_to_drop = ['lr-Adam','step','test_IoU','test_acc','test_fbeta','test_jaccard','test_loss','train_acc','val_acc']

# new_test_df = csv_file.drop(columns=columns_to_drop)



# for metric_name in new_test_df.columns[1:]:
#     fig, ax = plt.subplots(figsize=(7,5))
#     if metric_name[:5]=='train':
#         ax.set_xlabel('# epochs')

#         metric_id = metric_name.replace('train_','')    
#         if metric_id not in metric_id_list:
#             val_metric_name = metric_name.replace('train','val')
#             train_values = new_test_df[metric_name].astype('float32')
#             val_values = new_test_df[val_metric_name].astype('float32')
#             ax.plot(train_values,color='b',label=f'Validation {train_values[len(train_values)-1]:.3E}')
#             ax.plot(val_values,color='r',label=  f'Training    {val_values[len(val_values)-1]:.3E}')
#             if metric_id == 'fbeta':
#                 ax.set_title('F-1 evolution',fontname="Charter",weight='bold')
#             else:
#                 ax.set_title(f'{metric_id[0].upper()+metric_id[1:]} evolution',fontname="Charter",weight='bold')
            
#             ax.set_ylabel(f'{metric_id[0].upper()+metric_id[1:]}')
            
#             # plt.annotate('%0.2f' % train_values[len(train_values)-1], xy=(len(train_values)-1, train_values[len(train_values)-1]))
#             # plt.annotate('%0.2E' % train_values[len(train_values)-1], xy=(new_test_df['epoch'][len(new_test_df['epoch'])-1], train_values[len(train_values)-1]),xytext=(-40,5),textcoords='offset pixels')

#             # plt.annotate('%0.2E' % val_values[len(val_values)-1], xy=(new_test_df['epoch'][len(new_test_df['epoch'])-1], val_values[len(val_values)-1]),xytext=(-40,5),textcoords='offset pixels')
#             metric_id_list.append(metric_id)
#         else:
#             metric_id = None
#             plt.close(fig)
#     elif metric_name[:3]=='val':
#         ax.set_xlabel('# epochs')
        
#         metric_id = metric_name.replace('val_','')
#         if metric_id not in metric_id_list:

#             train_metric_name = metric_name.replace('val','train')
#             train_values = new_test_df[metric_name].astype('float32')
#             val_values = new_test_df[train_metric_name].astype('float32')

#             ax.plot(train_values,color='b',label=f'Validation {train_values[len(train_values)-1]:.3E}',fontname="Charter")
#             ax.plot(val_values,color='r',label=f'Training {val_values[len(val_values)-1]:.3E}',fontname="Charter")
#             if metric_id == 'fbeta':
#                 ax.set_title('F-1 evolution',fontname="Charter",weight='bold')
#             else:
#                 ax.set_title(f'{metric_id[0].upper()+metric_id[1:]} evolution',fontname="Charter",weight='bold')
            
#             ax.set_ylabel(f'{metric_id[0].upper()+metric_id[1:]}')

#             # plt.annotate('%0.2E' % train_values[len(train_values)-1], xy=(new_test_df['epoch'][len(new_test_df['epoch'])-1], train_values[len(train_values)-1]),xytext=(-40,5),textcoords='offset pixels')

#             # plt.annotate('%0.2E' % val_values[len(val_values)-1], xy=(new_test_df['epoch'][len(new_test_df['epoch'])-1], val_values[len(val_values)-1]),xytext=(-40,5),textcoords='offset pixels')
            
#             metric_id_list.append(metric_id)
#         else:
#             metric_id = None
#             plt.close(fig)
#     else:
#         print('ERROR')
#         plt.close(fig)


#     if metric_id:
#         ax.legend()
#         plt.savefig(os.path.join(plots_path,metric_id+'_evolution.png'))

#         plt.show()
#     # plt.savefig(os.path.join(model_path,file_name.replace('.csv','.png')))
            


In [ ]:
# test_df = combined_df.copy()

# columns_to_drop = ['lr-Adam','step','test_IoU','test_acc','test_fbeta','test_jaccard','test_loss','train_acc','val_acc']

# new_test_df = test_df.drop(columns=columns_to_drop)

# for column in new_test_df.columns[1:]:
#     fig, ax = plt.subplots()
#     ax.set_xlabel('# epochs')
#     # ax.set_title(column)
#     ax.set_title(f'{column} evolution')
#     ax.set_ylabel(column)
#     ax.plot(new_test_df['epoch'],new_test_df[column],color='k')


#     if column[-4:]=='loss':
#         plt.annotate('%0.2E' % new_test_df[column][len(new_test_df[column])-1], xy=(new_test_df['epoch'][len(new_test_df['epoch'])-1], new_test_df[column][len(new_test_df[column])-1]),xytext=(-50,10),textcoords='offset pixels')
#     else:
#         ax.set_ylim(0,1)
#         plt.annotate('%0.3E' % new_test_df[column][len(new_test_df[column])-1], xy=(new_test_df['epoch'][len(new_test_df['epoch'])-1], new_test_df[column][len(new_test_df[column])-1]),xytext=(-40,6),textcoords='offset pixels')


# # ax.yaxis.set_minor_locator(AutoMinorLocator(4))
# # print(new_test_df[column][len(new_test_df[column])-1])

In [ ]:
# import pandas as pd
# %matplotlib inline
# # %matplotlib widget
# import os

# import matplotlib as mpl
# mpl.rc('font',family='Charter')

# from matplotlib.ticker import AutoMinorLocator, MultipleLocator

# import matplotlib.pyplot as plt

# # model_path = os.path.join(os.getcwd(),'models','current_best_model')

# metric_id_list = []

# model_name = 'Unet'

# ENCODER = 'mobilenet_v2'

# file_name = os.path.join(os.getcwd(),'models',f'{model_name}_{ENCODER}_metrics_without_empty_masks',f'metrics_{model_name}_{ENCODER}.csv')
# plots_path = os.path.join(os.path.dirname(file_name),'plots')
# os.makedirs(plots_path,exist_ok=True)


# csv_file = pd.read_csv(os.path.join(file_name))

# columns_to_drop = ['lr-Adam','step','test_IoU','test_acc','test_fbeta','test_jaccard','test_loss','train_acc','val_acc']

# new_test_df = csv_file.drop(columns=columns_to_drop)

# # ax.yaxis.set_minor_locator(AutoMinorLocator(4))

# for metric_name in new_test_df.columns[1:]:
#     fig, ax = plt.subplots(figsize=(7,5))
#     if metric_name[:5]=='train':
#         ax.set_xlabel('# epochs')

#         metric_id = metric_name.replace('train_','')    
#         if metric_id not in metric_id_list:
#             val_metric_name = metric_name.replace('train','val')
#             train_values = new_test_df[metric_name].astype('float32')
#             val_values = new_test_df[val_metric_name].astype('float32')
#             ax.plot(train_values,color='b',label=f'Validation {train_values[len(train_values)-1]:.3E}')
#             ax.plot(val_values,color='r',label=  f'Training    {val_values[len(val_values)-1]:.3E}')
#             if metric_id == 'fbeta':
#                 ax.set_title('F-1 evolution',fontname="Charter")
#             else:
#                 ax.set_title(f'{metric_id[0].upper()+metric_id[1:]} evolution',fontname="Charter")
            
#             ax.set_ylabel(f'{metric_id[0].upper()+metric_id[1:]}')
            
#             # plt.annotate('%0.2f' % train_values[len(train_values)-1], xy=(len(train_values)-1, train_values[len(train_values)-1]))
#             # plt.annotate('%0.2E' % train_values[len(train_values)-1], xy=(new_test_df['epoch'][len(new_test_df['epoch'])-1], train_values[len(train_values)-1]),xytext=(-40,5),textcoords='offset pixels')

#             # plt.annotate('%0.2E' % val_values[len(val_values)-1], xy=(new_test_df['epoch'][len(new_test_df['epoch'])-1], val_values[len(val_values)-1]),xytext=(-40,5),textcoords='offset pixels')
#             metric_id_list.append(metric_id)
#         else:
#             metric_id = None
#             plt.close(fig)
#     elif metric_name[:3]=='val':
#         ax.set_xlabel('# epochs')
        
#         metric_id = metric_name.replace('val_','')
#         if metric_id not in metric_id_list:

#             train_metric_name = metric_name.replace('val','train')
#             train_values = new_test_df[metric_name].astype('float32')
#             val_values = new_test_df[train_metric_name].astype('float32')

#             ax.plot(train_values,color='b',label=f'Validation {train_values[len(train_values)-1]:.3E}',fontname="Charter")
#             ax.plot(val_values,color='r',label=f'Training {val_values[len(val_values)-1]:.3E}',fontname="Charter")
#             ax.set_title(f'{metric_id[0].upper()+metric_id[1:]} evolution',fontname="Charter")
#             ax.set_ylabel(f'{metric_id[0].upper()+metric_id[1:]}')

#             # plt.annotate('%0.2E' % train_values[len(train_values)-1], xy=(new_test_df['epoch'][len(new_test_df['epoch'])-1], train_values[len(train_values)-1]),xytext=(-40,5),textcoords='offset pixels')

#             # plt.annotate('%0.2E' % val_values[len(val_values)-1], xy=(new_test_df['epoch'][len(new_test_df['epoch'])-1], val_values[len(val_values)-1]),xytext=(-40,5),textcoords='offset pixels')
            
#             metric_id_list.append(metric_id)
#         else:
#             metric_id = None
#             plt.close(fig)
#     else:
#         print('ERROR')
#         plt.close(fig)


#     if metric_id:
#         ax.legend()
#         plt.savefig(os.path.join(plots_path,metric_id+'_evolution.png'))

#         plt.show()
#     # plt.savefig(os.path.join(model_path,file_name.replace('.csv','.png')))
            
